# 매출이 왜 감소하고 있는가?

## 1. Loading data

#### 1) 필요한 라이브러리들을 호출

In [1]:
#데이터 분석에 필요한 패키지들을 파이썬으로 호출
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#### 2) 필요 데이터 경로 지정

In [2]:
dauPath = os.path.expanduser("Week9Data/week9-dau.csv")
dpuPath = os.path.expanduser("Week9Data/week9-dpu.csv")
installPath = os.path.expanduser("Week9Data/week9-install.csv")

#### 3) 각 변수별로 데이터 할당 

In [3]:
dau = pd.read_csv(dauPath, sep=",")
dpu = pd.read_csv(dpuPath, sep=",")
install = pd.read_csv(installPath, sep=",")

FileNotFoundError: File b'Week9Data/week9-dau.csv' does not exist

#### 4) 호출된 데이터 정보 확인

In [4]:
dau.info()
dau[:5]

NameError: name 'dau' is not defined

In [5]:
dpu.info()
dpu[:5]

NameError: name 'dpu' is not defined

In [6]:
install.info()
install[:5]

NameError: name 'install' is not defined

# 2. Data cleaning

#### 1) 각 데이터들을 결합
merge(): 키를 하나 이상 사용해 데이터 프레임의 각 열들을 합치는 기능을 가지고 있음
- merge()인자 'how=  ' 결합 유형을 정의하는 인자값으로 default는 inner 유형 (완벽히 같은 값만 호출/교집합), outer 유형은 합집합

In [ ]:
#Install데이터에 Dau 데이터를 결합하기
InstallDau = install.merge(dau, on=['user_id', 'app_name'])

In [ ]:
InstallDau.info()
InstallDau[:5]

In [ ]:
#위 데이터에 다시 DPU데이터를 결합하기
InstallDauPayment = InstallDau.merge(dpu, on=["log_date", "app_name", "user_id"], how="outer")

In [ ]:
InstallDauPayment.info()
InstallDauPayment[:5]

#### 2) 결측치(NaN)를 0으로 대체

In [ ]:
#결측치를 0으로 대체
InstallDauPayment = InstallDauPayment.fillna(0)

In [ ]:
InstallDauPayment.info()
InstallDauPayment[:5]

#### 3) 월별 정렬을 위한 문자처리

In [ ]:
#log_date열(serise)를 기존 log_date의 앞에서 7번째 값으로 치환 
InstallDauPayment['log_date'] = InstallDauPayment['log_date'].str[:7]

#Install_date열(serise)를 기존 Install_date의 앞에서 7번째 값으로 치환 
InstallDauPayment['install_date'] = InstallDauPayment['install_date'].str[:7]

#변환된 결과 확인
InstallDauPayment[:5]

#### 4) 신규 유저 분류

In [ ]:
#데이터프레임에 User_type 시리즈를 만들고 
#Install날짜와 접속일자가 같으면 Install 고객(신규유저), 그렇지 않으면 existing 고객(기존 고객)으로 표현
InstallDauPayment["user_type"] = np.where(InstallDauPayment['log_date'] == InstallDauPayment['install_date'], 'install', 'existing')

In [ ]:
InstallDauPayment[:5]

# 3-4. Data analysis & Visualization

### 1-1. 메인분석: 월별 유저 현황

#### 1) Seaborn을 이용한 시각화

In [ ]:
#seaborn 그래프 사이즈 고정
sns.set(rc={"figure.figsize":(6, 6)})

#seaborn 라이브러리 중 countplot 사용
sns.countplot(data=InstallDauPayment, x="log_date", hue="user_type")

#### 2) pandas 자체 함수를 이용한 시각화

In [ ]:
#payment를 log_date와 user_type을 기준으로 분류
grouped = InstallDauPayment['payment'].groupby([InstallDauPayment["log_date"],InstallDauPayment["user_type"]]).sum()

#grouped 호출
grouped 

In [ ]:
#plot 함수를 통한 시각화
grouped.unstack().plot(kind="bar", stacked=True, color=["c", "m"], grid=False, figsize=[6, 6], title="Payment Graph")

### 1-2. 추가 분석: 신규유저의 payment 현황

#### 신규유저만 시각화

In [ ]:
#InstallDauPayment 데이터프레임을 "user_type"인덱스로 정렬
df = InstallDauPayment.set_index("user_type")

#df데이터프레임 중 기존 고객'행'을 제거
df.drop("existing", inplace = True)

In [ ]:
#df데이터프레임을 'payment' 인덱스로 정렬
df = df.set_index("payment")

In [ ]:
#df데이터프레임 중 지불액 0원 제거
df.drop("0", inplace = True)

#df데이터프레임을 새 인덱스로 정렬
df = df.reset_index()

In [ ]:
#2016-06월, 2016-07월 기준의 데이터프레임을 호출하고 x1, x2변수로 저장
x1=df.set_index("log_date").loc["2013-06"].payment.as_matrix()
x2=df.set_index("log_date").loc["2013-07"].payment.as_matrix()

#시각화를 위한 data를 만들기 위해 변수 x1, x2를 numpy.array 형태로 저장
data=np.array([x1, x2])

#시각화 스타일('seaborn-deep')지정
plt.style.use("seaborn-deep")

#그래프에 사용되는 변수들 정의
bins=np.linspace(0, 30000, 10) 
plt.xlabel("Payment")
plt.ylabel("count")

#시각화 수행
plt.hist(data, bins, label=['2013-06', '2013-07'])
plt.legend(loc='upper right')